In [ ]:
pip install -qr requirements.txt

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [ ]:
df = pd.read_csv('data/expert_opinions.csv')

experts_count = len(df)
experts_answers = {}
types = [int(i) for i in df.keys()]
options = set()
for i in range(experts_count):
    experts_answers.update({i: []})
    for k in df:
        answer = df[k][i]
        experts_answers[i].append(answer)

        options.add(answer)

print(f'Множество значений: {types}')
print(f'Термы: {options}')

In [ ]:
for i in types:
    print(f'{i:<3} ', end='')
print()
print('----' * len(types))
for k in experts_answers:
    for i in options:
        for j in experts_answers[k]:
            if i == j:
                print('+   ', end='')
            else:
                print('    ', end='')
        print(i)
    print('----' * len(types))

In [ ]:
membership_table = {j: [0 for i in range(len(types))] for j in options}
for k in experts_answers:
    for i in range(len(experts_answers[k])):
        answer = experts_answers[k][i]
        membership_table[answer][i] += 1

for i in types:
    print(f'{i:<5} ', end='')
print()
print('------' * len(types))
for k in membership_table:
    for i in membership_table[k]:
        print(f'{i:<5} ', end='')
    print()
    for i in membership_table[k]:
        print(f'{i / experts_count:<5} ', end='')
    print(k)
    print('------' * len(types))

for k in membership_table:
    plt.plot(
        types,
        [i / experts_count for i in membership_table[k]],
        label=k
    )
plt.legend()
plt.show()


In [ ]:
category_tree = json.load(open('data/category_tree.json'))

def print_category_tree(Node=category_tree, prefix=''):
    level_size = len(Node)
    for category in Node:
        level_size -= 1

        if level_size == 0:
            angle = '└──'
            new_prefix = f'{prefix}   '
        else:
            angle = '├──'
            new_prefix = f'{prefix}│  '
        
        print(f'{prefix}{angle}{category}')
        print_category_tree(Node[category], new_prefix)


print_category_tree()

In [ ]:
def find_categories_proximity(lhs, rhs):
    def find_category(category, Node=category_tree):
        assert Node is not None

        for i in Node:
            if category == i:
                return [category]
            else:
                in_sub_tree = find_category(category, Node[i])
                if in_sub_tree is not None:
                    return [i] + in_sub_tree
            
        return None
    
    lhs = find_category(lhs)
    rhs = find_category(rhs)

    for i in range(len(min(lhs, rhs))):
        if lhs[i] != rhs[i]:
            return 1 / (len(lhs) - i + len(rhs) - i + 1)
        
    return 1